In [1]:
!pip install datasets transformers -q


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


# Create Dataset for Instruction Backtranslation

We load the [andersonbcdefg/guanaco-ai-filtered](https://huggingface.co/datasets/andersonbcdefg/guanaco-ai-filtered) dataset which is a carefully curated subset of the Open Assistant dataset containing only the highest-rated conversation paths from the original dataset, specifically designed for training models to generate instructions based on responses. It has been filtered to:

* Remove conversations in languages other than English using a fasttext classifier
* Remove conversations where Open Assistant is mentioned, as people training their own chatbots likely do not want their chatbot to think it is named OpenAssistant.

We additionally pre-process the data:

* To include Single-turn conversations only
* Formatted with Llama 2 chat model prompt instruction tokens
* Response and instruction roles are swapped for reverse fine-tuning

In [ ]:
from datasets import load_dataset, Dataset

def transform_conversation(example):
    segments = example['text'].split('###')
    if len(segments) != 3:
        return None

    human = segments[1].replace('Human:', '').strip()
    assistant = segments[2].replace('Assistant:', '').strip()

    if 'Human:' not in segments[1] or 'Assistant:' not in segments[2]:
        return None

    return {'text': f'<s>[INST] {assistant} [/INST] {human} </s>'}

# Load the dataset
dataset = load_dataset('andersonbcdefg/guanaco-ai-filtered')

# Apply the transformation and filter
transformed_data = [
    transform_conversation(example)
    for example in dataset['train']
    if transform_conversation(example) is not None
]

# Create a new dataset from the transformed data
transformed_dataset = Dataset.from_list(transformed_data)

# Push the Instruction BackTranslation dataset to HF¶

In [ ]:
transformed_dataset.push_to_hub("guanaco-llama2-reverse-instruct")